In [ ]:
#default_exp s3_scrape

In [3]:
#export

import boto3
import pandas as pd
import os
from PIL import Image
import json

In [2]:
#export

def copy_jpgs(json_path, source_bucket, target_bucket):
    s3_client = boto3.client('s3')
    
    images = 0
    
    with open(json_path, encoding="utf-8") as jsonfile:
        data = json.load(jsonfile)
        
    for volume in data["volumes"]:
        print("Now working on " + volume["identifier"])
        for image in volume["images"]:            
            #copy_source = {"Bucket": source_bucket, "Key": volume["s3_path"] + "/JPG/" + str(image["file_name"]) + ".JPG"}
            try:
                s3_client.download_file(source_bucket, volume["s3_path"] + "/JPG/" + str(image["file_name"]) + ".JPG", "temp.jpg")
            except:
                s3_client.download_file(source_bucket, volume["s3_path"] + "/JPG/" + str(image["file_name"]) + ".jpg", "temp.jpg")            
            image_number = str(image["file_name"] - 1000)
            padded_number = '0' * (4 - len(image_number)) + image_number
            #s3_client.copy(copy_source, target_bucket, str(volume["identifier"]) + '-' + padded_number + ".jpg", ExtraArgs={'ContentType': "image/jpeg", 'Metadata': {"x-amz-meta-width": str(image["width"]), "x-amz-meta-height": str(image["height"])}})
            s3_client.upload_file("temp.jpg", target_bucket, str(volume["identifier"]) + '-' + padded_number + ".jpg", ExtraArgs={'ContentType': "image/jpeg", 'Metadata': {"width": str(image["width"]), "height": str(image["height"])}})
            images += 1
            
    os.remove("temp.jpg")
            
    return str(images) + " images copied from " + source_bucket + " to " + target_bucket

In [3]:
#no_test

copy_jpgs("cuba.json", "ssda-assets", "ssda-production-jpgs")

Now working on 20387
Now working on 20475
Now working on 21251
Now working on 21252
Now working on 21954
Now working on 21963
Now working on 22029
Now working on 22152
Now working on 22370
Now working on 22614
Now working on 24233
Now working on 24335
Now working on 24352
Now working on 24473
Now working on 24474
Now working on 24671
Now working on 24672
Now working on 24673
Now working on 24674
Now working on 25291
Now working on 25292
Now working on 25344
Now working on 25470
Now working on 26267
Now working on 26268
Now working on 26269
Now working on 26718
Now working on 27140
Now working on 27576
Now working on 27577
Now working on 27578
Now working on 27579
Now working on 27580
Now working on 27581
Now working on 306662
Now working on 324571
Now working on 324890
Now working on 325192
Now working on 325601
Now working on 326363
Now working on 326519
Now working on 326801
Now working on 327049
Now working on 327399
Now working on 327693
Now working on 153874
Now working on 154082


'14403 images copied from ssda-assets to ssda-production-jpgs'

In [9]:
#export

def build_volume_records(json_path, target_bucket):
    s3_resource = boto3.resource('s3')
    s3_client = boto3.client('s3')
    #bucket = s3_resource.Bucket(target_bucket)    
    
    volumes = 0
    
    with open(json_path, encoding="utf-8") as jsonfile:
        data = json.load(jsonfile)
        
    for volume in data["volumes"]:
        keys_to_delete = ["s3_path", "contributor"]
        keys_to_add = []
        values_to_add = []
        for key in volume:
            if key in keys_to_delete:                
                continue
            elif (type(volume[key]) == list) and ((key == "creator") or (key == "format")):                
                temp = ''
                for element in volume[key]:
                    temp += element + "; "
                temp = temp[:len(temp) - 2]
                volume[key] = temp
            elif key == "date":
                if volume[key]["start year"] == None:
                    start_date = "1500"
                else:
                    start_date = str(volume[key]["start year"])
                if volume[key]["start month"] != None:
                    str_start_month = str(volume[key]["start month"])
                    start_date += '-' + '0' * (2 - len(str_start_month)) + str_start_month
                    if volume[key]["start day"] != None:
                        str_start_day = str(volume[key]["start day"])
                        start_date += '-' + '0' * (2 - len(str_start_day)) + str_start_day
                if volume[key]["end year"] == None:
                    end_date = "2000"
                else:
                    end_date = str(volume[key]["end year"])
                if volume[key]["end month"] != None:
                    str_end_month = str(volume[key]["end month"])
                    end_date += '-' + '0' * (2 - len(str_end_month)) + str_end_month
                    if volume[key]["end day"] != None:
                        str_end_day = str(volume[key]["end day"])
                        end_date += '-' + '0' * (2 - len(str_end_day)) + str_end_day                
                keys_to_delete.append(key)                
            elif type(volume[key]) == dict:
                for key_key in volume[key]:
                    keys_to_add.append(key_key)
                    values_to_add.append(volume[key][key_key])
                keys_to_delete.append(key)
            elif volume[key] == None:
                volume[key] = ''
                
        for i in range(len(keys_to_add)):
            if values_to_add[i] == None:
                values_to_add[i] = ''
            
            if (keys_to_add[i] == "coords") and (values_to_add[i] != ''):
                coords = values_to_add[i].split(',')
                volume[keys_to_add[i]] = '(' + coords[0] + ", " + coords[1] + ')'
                continue
            elif keys_to_add[i] == "coords":
                volume[keys_to_add[i]] = "(0,0)"
                continue
                
            volume[keys_to_add[i]] = values_to_add[i]
            
        volume["start_date"] = start_date
        volume["end_date"] = end_date
        
        for key in keys_to_delete:            
            del volume[key]
                    
        with open("temp.json", 'w', encoding="utf-8") as outfile:
            json.dump(volume, outfile)               
        s3_client.upload_file("temp.json", target_bucket, str(volume["identifier"]) + ".json", ExtraArgs={'ContentType': "application/json"})
        volumes += 1
        
    return "Metadata for " + str(volumes) + " volumes uploaded to S3."

In [11]:
#no_test

build_volume_records("us-vol.json", "ssda-volume-metadata")

'Metadata for 30 volumes uploaded to S3.'

In [14]:
#export

def build_cloudsearch_batch(json_path, output_dir):
    
    s3_resource = boto3.resource('s3')
    s3_client = boto3.client('s3')
    #bucket = s3_resource.Bucket(target_bucket)    
    
    volumes = 0
    batch = []
    
    with open(json_path, encoding="utf-8") as jsonfile:
        data = json.load(jsonfile)
        
    for volume in data["volumes"]:
        keys_to_delete = ["s3_path", "contributor"]
        keys_to_add = []
        values_to_add = []
        for key in volume:
            if key in keys_to_delete:                
                continue
            elif (type(volume[key]) == list) and ((key == "creator") or (key == "format")):                
                temp = ''
                for element in volume[key]:
                    temp += element + "; "
                temp = temp[:len(temp) - 2]
                volume[key] = temp
            elif key == "date":
                if volume[key]["start year"] == None:
                    start_date = "1500"
                else:
                    start_date = str(volume[key]["start year"])
                if volume[key]["start month"] != None:
                    str_start_month = str(volume[key]["start month"])
                    start_date += '-' + '0' * (2 - len(str_start_month)) + str_start_month
                    if volume[key]["start day"] != None:
                        str_start_day = str(volume[key]["start day"])
                        start_date += '-' + '0' * (2 - len(str_start_day)) + str_start_day
                if volume[key]["end year"] == None:
                    end_date = "2000"
                else:
                    end_date = str(volume[key]["end year"])
                if volume[key]["end month"] != None:
                    str_end_month = str(volume[key]["end month"])
                    end_date += '-' + '0' * (2 - len(str_end_month)) + str_end_month
                    if volume[key]["end day"] != None:
                        str_end_day = str(volume[key]["end day"])
                        end_date += '-' + '0' * (2 - len(str_end_day)) + str_end_day                
                keys_to_delete.append(key)                
            elif type(volume[key]) == dict:
                for key_key in volume[key]:
                    keys_to_add.append(key_key)
                    values_to_add.append(volume[key][key_key])
                keys_to_delete.append(key)
            elif volume[key] == None:
                volume[key] = ''
                
        for i in range(len(keys_to_add)):
            if values_to_add[i] == None:
                values_to_add[i] = ''
            
            if (keys_to_add[i] == "coords") and (values_to_add[i] != ''):
                coords = values_to_add[i].split(',')
                volume[keys_to_add[i]] = coords[0] + ", " + coords[1]
                continue
            elif keys_to_add[i] == "coords":
                volume[keys_to_add[i]] = "0, 0"
                continue
                
            volume[keys_to_add[i]] = values_to_add[i]
            
        if len(start_date) == 10:
            volume["start_date"] = start_date + "T00:00:00Z"
        elif len(start_date) == 7:
            volume["start_date"] = start_date + "-01T00:00:00Z"
        else:
            volume["start_date"] = start_date + "-01-01T00:00:00Z"
        
        date_parts = end_date.split('-')
        long_months = ["01", "03", "05", "07", "08", "10", "12"]
        if len(date_parts) == 2:
            if date_parts[1] in long_months:
                end_month_length = "31"
            elif date_parts[1] == "02":
                end_month_length = "28"
            else:
                end_month_length = "30"
        
        if len(end_date) == 10:
            volume["end_date"] = end_date + "T23:59:59Z"
        elif len(end_date) == 7:
            volume["end_date"] = end_date + end_month_length + "T23:59:59Z"
        else:
            volume["end_date"] = end_date + "-12-31T23:59:59Z"
        
        for key in keys_to_delete:            
            del volume[key]
                    
        batch_record = {"type": "add", "id": str(volume["identifier"]), "fields": volume}
        batch.append(batch_record)
        volumes += 1
        
    with open(output_dir, 'w', encoding="utf-8") as outfile:
        json.dump(batch, outfile)   
        
    return "Metadata for " + str(volumes) + " volumes combined in a CloudSearch batch request available at " + output_dir

In [15]:
#no_test

build_cloudsearch_batch("us-vol.json", "us-cs-batch.json")

'Metadata for 30 volumes combined in a CloudSearch batch request available at us-cs-batch.json'

In [ ]:
#export

def scrape_bucket(bucket_name, prefix=None):
    s3_resource = boto3.resource('s3')
    s3_client = boto3.client('s3')
    bucket = s3_resource.Bucket(bucket_name)
    
    volume_ids = []
    titles = []
    volume_roots = []
    image_counts = []
    has_jpg = []
    has_tif = []
    has_other = []
    other = []
    has_pdf = []
    has_metadata = []
    volume_metadata = []
    
    folders = ["jpg", "tif", "metadata"]   
    
    for obj in bucket.objects.filter(Prefix = prefix):    
        if (len(obj.key.split('/')) >= 4) and (obj.key.split('/')[3].isdigit()) and (obj.key.split('/')[3] != '') and (obj.key.split('/')[3] not in volume_ids):
            volume_ids.append(obj.key.split('/')[3])
            volume_root = obj.key[:obj.key.find('/', obj.key.find(obj.key.split('/')[3]))]
            volume_roots.append(volume_root)            
            has_metadata.append(False)
            has_pdf.append(False)
            has_jpg.append(False)
            has_tif.append(False)
            has_other.append(False)
            other.append(None)
            for volume_obj in bucket.objects.filter(Prefix = volume_root):
                if "DC.xml" in volume_obj.key:                    
                    has_metadata[-1] = True
                    s3_client.download_file(bucket.name, volume_obj.key, "temp.xml")
                    vol_dict = ssda_volume_xml_to_dict("temp.xml", volume_root)
                    volume_metadata.append(vol_dict)
                    if "title" in vol_dict:
                        titles.append(vol_dict["title"])
                    else:
                        titles.append("no title")
                    os.remove("temp.xml")
                elif "pdf" in volume_obj.key.lower():
                    has_pdf[-1] = True
                elif (has_jpg[-1] == False) and (volume_obj.key.lower().split('/')[4] == "jpg"):
                    has_jpg[-1] = True
                elif (has_tif[-1] == False) and (volume_obj.key.lower().split('/')[4] == "tif"):
                    has_tif[-1] = True
                elif (len(volume_obj.key.split('/')) > 5) and (volume_obj.key.lower().split('/')[4] not in folders) and ((other[-1] == None) or (volume_obj.key.lower().split('/')[4] not in other[-1])):
                    has_other[-1] = True
                    if other[-1] == None:
                        other[-1] = volume_obj.key.lower().split('/')[4]
                    else:
                        other[-1] = other[-1] + '|' + volume_obj.key.lower().split('/')[4]
                
            image_metadata = []
            prod_imgs = None
            if has_jpg[-1]:
                prod_imgs = "jpg"
            elif has_tif[-1]:
                prod_imgs = "tif"
                
            if prod_imgs == None:
                volume_metadata[-1]["images"] = []
                image_counts.append(0)
            else:
                bad_images = 0
                for image_obj in bucket.objects.filter(Prefix = volume_root + '/' + prod_imgs.upper()):
                    if ('.' + prod_imgs) in image_obj.key.lower():
                        file_name = image_obj.key[image_obj.key.rfind('/') + 1:image_obj.key.rfind('.')]
                        if not file_name.isdigit():
                            print("found bad image file name at " + image_obj.key)
                            bad_images += 1
                            continue
                        extension = image_obj.key[image_obj.key.rfind('.') + 1:]
                        temp_path = file_name + '.' + extension
                        s3_client.download_file(bucket.name, image_obj.key, temp_path)
                        im = Image.open(temp_path)
                        width, height = im.size
                        im.close()
                        os.remove(temp_path)
                        image = {"file_name": int(file_name), "extension": extension, "height": height, "width": width}
                        image_metadata.append(image)
                volume_metadata[-1]["images"] = image_metadata
                image_counts.append(len(image_metadata) + bad_images)           
            
            print("Completed " + titles[-1])
        elif (len(obj.key.split('/')) >= 5) and (not obj.key.split('/')[3].isdigit()) and (obj.key.split('/')[4] != '') and (obj.key.split('/')[4] not in volume_ids):
            volume_ids.append(obj.key.split('/')[4])
            volume_root = obj.key[:obj.key.find('/', obj.key.find(obj.key.split('/')[4]))]
            volume_roots.append(volume_root)            
            has_metadata.append(False)
            has_pdf.append(False)
            has_jpg.append(False)
            has_tif.append(False)
            has_other.append(False)
            other.append(None)
            for volume_obj in bucket.objects.filter(Prefix = volume_root):
                if "DC.xml" in volume_obj.key:                    
                    has_metadata[-1] = True
                    s3_client.download_file(bucket.name, volume_obj.key, "temp.xml")
                    vol_dict = ssda_volume_xml_to_dict("temp.xml", volume_root)
                    volume_metadata.append(vol_dict)
                    if ("title" in vol_dict) and (vol_dict["title"] != None):
                        titles.append(vol_dict["title"])
                    else:
                        titles.append("no title")
                    os.remove("temp.xml")
                elif "pdf" in volume_obj.key.lower():
                    has_pdf[-1] = True
                elif (has_jpg[-1] == False) and (volume_obj.key.lower().split('/')[5] == "jpg"):
                    has_jpg[-1] = True
                elif (has_tif[-1] == False) and (volume_obj.key.lower().split('/')[5] == "tif"):
                    has_tif[-1] = True
                elif (len(volume_obj.key.split('/')) > 6) and (volume_obj.key.lower().split('/')[5] not in folders) and ((other[-1] == None) or (volume_obj.key.lower().split('/')[5] not in other[-1])):
                    has_other[-1] = True
                    if other[-1] == None:
                        other[-1] = volume_obj.key.lower().split('/')[5]
                    else:
                        other[-1] = other[-1] + '|' + volume_obj.key.lower().split('/')[5]
                        
            if has_metadata[-1] == False:
                titles.append("no title")
                print("Failed to find metadata for " + volume_root)
                
            image_metadata = []
            prod_imgs = None
            if has_jpg[-1]:
                prod_imgs = "jpg"
            elif has_tif[-1]:
                prod_imgs = "tif"
                
            if prod_imgs == None:
                volume_metadata[-1]["images"] = []
                image_counts.append(0)
            else:
                bad_images = 0
                for image_obj in bucket.objects.filter(Prefix = volume_root + '/' + prod_imgs.upper()):
                    if ('.' + prod_imgs) in image_obj.key.lower():
                        file_name = image_obj.key[image_obj.key.rfind('/') + 1:image_obj.key.rfind('.')]
                        if not file_name.isdigit():
                            print("found incorrect image file name at " + image_obj.key)
                            bad_images += 1
                            continue
                        extension = image_obj.key[image_obj.key.rfind('.') + 1:]
                        temp_path = file_name + '.' + extension
                        s3_client.download_file(bucket.name, image_obj.key, temp_path)
                        try:
                            im = Image.open(temp_path)
                            width, height = im.size
                            im.close()                        
                            image = {"file_name": int(file_name), "extension": extension, "height": height, "width": width}
                            image_metadata.append(image)
                        except:
                            print("found bad image file at " + image_obj.key)
                            bad_images += 1
                        os.remove(temp_path)
                volume_metadata[-1]["images"] = image_metadata
                image_counts.append(len(image_metadata) + bad_images)           
            
            try:
                print("Completed " + titles[-1])
            except:
                print("Completed")
                print(titles[-1])
                
    volumes_dict = {"id": volume_ids, "title": titles, "images": image_counts, "s3 root": volume_roots, "metadata": has_metadata, "has pdf": has_pdf, "has jpg": has_jpg, "has tif": has_tif, "has other": has_other, "other": other}
    volumes_df = pd.DataFrame.from_dict(volumes_dict)
    
    return volumes_df, volume_metadata  

In [ ]:
#export

def ssda_volume_xml_to_dict(volume_xml, s3_path):
    import xml.etree.ElementTree as ET
    tree = ET.parse(volume_xml)
    root = tree.getroot()
    volume_dict = {}
    volume_dict["s3_path"] = s3_path
    for item in root:
        if "{http://purl.org/dc/elements/1.1/}" in item.tag:
            item.tag = item.tag[item.tag.find('}') + 1:]
        if item.text == None:
            if item.tag not in volume_dict:
                volume_dict[item.tag] = None
            continue
        if item.text[0] == ' ':
            item.text = item.text[1:]        
        if item.tag == "subject":
            if "subject" in volume_dict:
                volume_dict["subject"].append(item.text.split("--"))
            else:
                volume_dict["subject"] = item.text.split("--")
        elif item.tag == "title":
            volume_dict["title"] = item.text
        elif item.tag == "contributor":
            if (item.text.find('(') != -1) and (item.text.find(')') != -1):
                name = item.text[:item.text.find('(')]
                role = item.text[item.text.find('(') + 1:item.text.find(')')]
            else:
                continue
            if "contributor" in volume_dict:
                volume_dict["contributor"].append({"name": name, "role": role})
            else:
                volume_dict["contributor"] = [{"name": name, "role": role}]
        elif item.tag == "identifier":
            volume_dict["identifier"] = item.text[item.text.find(':') + 1:]
        elif item.tag == "coverage":
            if ('.' in item.text) and (',' in item.text) and ("Archives" not in item.text):
                if "coverage" in volume_dict:
                    volume_dict["coverage"]["coords"] = item.text
                else:
                    volume_dict["coverage"] = {"coords": item.text}
            elif "--" in item.text:
                places = item.text.split("--")
                if len(places) == 4:
                    if "coverage" in volume_dict:
                        volume_dict["coverage"]["country"] = places[1]
                        volume_dict["coverage"]["state"] = places[2]
                        volume_dict["coverage"]["city"] = places[3]
                    else:
                        volume_dict["coverage"] = {"country": places[1], "state": places[2], "city": places[3]}                
        elif item.tag == "source":
            if "coverage" in volume_dict:
                volume_dict["coverage"]["institution"] = item.text
            else:
                volume_dict["coverage"] = {"institution": item.text}
        elif ((item.tag == "type") and (item.text == "Text")) or (item.tag == "rights"):
            continue
        elif (item.tag == "creator") and (';' in item.text):            
            creators = item.text.split(';')
            for creator in creators:                
                if (len(creator) > 1) and (creator[0] == ' '):
                    creator = creator[1:]
                if "creator" in volume_dict:
                    volume_dict["creator"].append(creator)
                else:
                    volume_dict["creator"] = [creator]
        elif (item.tag == "language") and (';' in item.text):
            languages = item.text.split(';')
            for language in languages:
                if (len(language) > 1) and (language[0] == ' '):
                    language = language[1:]
                if "language" in volume_dict:
                    volume_dict["language"].append(language)
                else:
                    volume_dict["language"] = [language]
        else:            
            if item.tag in volume_dict:                
                volume_dict[item.tag].append(item.text)                
            else:
                volume_dict[item.tag] = [item.text]       
        
    if "coverage" not in volume_dict:
        volume_dict["coverage"] = {}               
        volume_dict["coverage"]["country"] = volume_dict["s3_path"].split('/')[0].replace('_', ' ')
        volume_dict["coverage"]["state"] = volume_dict["s3_path"].split('/')[1].replace('_', ' ')
        volume_dict["coverage"]["city"] = volume_dict["s3_path"].split('/')[2].replace('_', ' ')
        volume_dict["coverage"]["institution"] = volume_dict["s3_path"].split('/')[3].replace('_', ' ')
    elif "institution" not in volume_dict["coverage"]:
        volume_dict["coverage"]["institution"] = volume_dict["s3_path"].split('/')[3].replace('_', ' ')
        
    return volume_dict

In [ ]:
#no_test

prefixes = ["Colombia/Chocó/Quibdó/Notaria_Primera_de_Quibdó/56975"]

for pref in prefixes:
    df, metadata = scrape_bucket("ssda-assets", prefix=pref)
    pref = pref.replace('/', '_')
    pref = pref[:-1]
    df.to_csv(pref.lower() + ".csv", index = False)
    with open(pref.lower() + ".json", 'w', encoding="utf-8") as outfile:
        outfile.write('{\n\"volumes\": \n')
        json.dump(metadata, outfile)
        outfile.write('}')
    print("Finished working on " + pref)

Completed Libro de Venta de Esclavo
Finished working on Colombia_Chocó_Quibdó_Notaria_Primera_de_Quibdó_5697


In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted manifest_generation.ipynb.
Converted s3_scrape.ipynb.
Converted s3_scrape_dev.ipynb.
